# 텍스트 다중분류 워드 임베딩(한글 네이버)

In [0]:
# 구글 드라이브와 연결

#from google.colab import auth
#auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# 경로 설정
naver_dir = '/content/gdrive/My Drive/pytest/네이버뉴스-생활문화_다중/'

!ls '/content/gdrive/My Drive/pytest/네이버뉴스-생활문화_다중/'

test  train


In [0]:
# 형태소분석기 관련 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk

!pip install JPype1==0.7.4
!pip install rhinoMorph

In [0]:
# Data Loading Preparation
import os

train_dir = os.path.join(naver_dir, 'train')
test_dir = os.path.join(naver_dir, 'test')

label_types = ['건강정보', '공연전시', '도로교통', '여행레저', '음식맛집', '자동차시승기', '종교', '책', '패션뷰티']

In [0]:
# text와 labels 리스트를 만드는 함수를 작성한다
def make_texts_labels(dir, label_types, encode='utf8'):    	# texts와 labels 리스트를 만드는 함수
  texts = []
  labels = []
  
  for label_type in label_types:
    dir_name = os.path.join(dir, label_type) 		  # 각각의 label 폴더에 접근한다
    for fname in os.listdir(dir_name):
      if fname[-4:] == '.txt':                    # 마지막 4 글자가 .txt 로 끝나는지를 확인한다
        f = open(os.path.join(dir_name, fname), encoding=encode)
        texts.append(f.read())                    # 텍스트를 읽어서 texts 리스트에 연결한다
        f.close()
        
        if label_type == label_types[0]:          # 만약 현재 폴더가 '건강정보' 폴더라면
          labels.append(0)                        # texts와 같은 순서의 labels 리스트에는 0을 저장한다
        elif label_type == label_types[1]:
          labels.append(1)
        elif label_type == label_types[2]:
          labels.append(2)
        elif label_type == label_types[3]:
          labels.append(3)
        elif label_type == label_types[4]:
          labels.append(4)
        elif label_type == label_types[5]:
          labels.append(5)
        elif label_type == label_types[6]:
          labels.append(6)
        elif label_type == label_types[7]:
          labels.append(7)
        elif label_type == label_types[8]:
          labels.append(8)
          
  return texts, labels

In [0]:
# Train Data Loading
# 훈련 데이터의 텍스트와 라벨을 받아온다

texts, labels = make_texts_labels(train_dir, label_types)

print('texts 0:', texts[0])
print('texts len:', len(texts))
print('labels 0:', labels[0])
print('labels len:', len(labels))

texts 0: 자폐증, ADHD 등의 뇌신경 보호 및 치료제 개발에 주력 : 네이버 뉴스







[스포츠서울 신재유기자] 바이오 의약품 연구에 특화된 벤처 기업 (주)뉴로벤티(대표 신찬영)가 자폐범주성장애(자폐증)와 주의력결핍과잉행동증후군(ADHD) 치료제 개발 분야에 도전장을 냈다. (주)뉴로벤티 신찬영 대표신찬영 대표는 건국대 의학전문대학원 교수로서 정신질환 중 유일하게 아직 확실한 치료약이 없는 자폐증에 주목하고 전문 치료제를 개발하고자 2015년 이 회사를 설립했다. (주)뉴로벤티 연구팀은 이온 통로형 글루탐산 수용체인 iGluR를 억제하는 치료제 개발에 매진해 iGluR 억제제 후보 물질에 대한 기술 권리를 국내외에서 확보하고 기전 연구를 진행했다. 또 다양한 자폐 동물 모델을 통해 자폐 동물들의 사회성과 사회 선택성을 측정해 자폐범주성장애 치료제를 검색할 수 있는 시스템을 구축하고 자폐증에 대한 다중 작용점 치료 효능 평가 연구를 했다. 그 결과 iGluR 억제제가 자폐 핵심 증상인 사회성 장애와 반복·과잉 행동을 개선하고 경련 등 동반 증상을 효과적으로 조절하며 두뇌 흥분성을 조절함으로서 사회성 결여를 개선한다는 것을 확인했다. 또한, ADHD 환자의 20-25%가 각성제 타입의약품에 반응하지 않고 다양한 부작용 발생으로 안전성 문제가 대두되고 있는데 현 상황에서는 동반 증상 조절과 다중 타킷 약물 개발이 주요 화두로 떠오르고 있는 점에 착안하여 세가지 작용기전으로 세가지의 핵심증상을 세가지 부작용 없이 치료할 수 있는 물질 개발을 아울러 진행하고 있다. 중소벤처기업부 ‘팁스(TIPS)’ 프로그램의 바이오 특화 투자관련 여섯 번째 투자 기업으로 선정된 (주)뉴로벤티는 2021년 자폐증 신약 후보 물질의 임상 진입을 목표로 연구에 진력한다. 아울러 자폐증 치료제를 제품화하고자 해외 파트너, 협력사, 투자자들과 국내외 기업에 기술을 이전하는 사업화 전략을 수립하는 중이다. 신 대표는 “4차 산업 혁명 시대에 부응하는 개인 맞춤형 의약품, 건강기

In [0]:
# Analyzing Morphology
# 한글은 tokenizing을 위해서 먼저 형태소분석을 해야 한다
import rhinoMorph
rn = rhinoMorph.startRhino()                         		

# 리스트 컴프리헨션으로 실질형태소만을 리스트로 가져온다
texts = [rhinoMorph.onlyMorph_list(rn, sentence, pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'], eomi=False) for sentence in texts]

print(texts)
print(texts[0])

filepath:  /usr/local/lib/python3.6/dist-packages
classpath:  /usr/local/lib/python3.6/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
[['자폐증', '등', '뇌신경', '보호', '및', '치료제', '개발', '주력', '네이버', '뉴스', '스포츠서울', '신', '재', '유기', '바이오', '의약품', '연구', '특화되', '벤처', '기업', '주', '뉴', '로벤', '티', '대표', '신찬영', '가', '자폐', '범주', '성장', '자폐증', '와', '주의력', '결핍', '잉', '증후', '치료제', '개발', '분야', '도전장', '내', '주', '뉴', '로벤', '티', '신찬영', '대표', '신찬영', '대표', '건국대', '의학', '전문', '대학원', '교수', '정신', '질환', '유일하', '아직', '확실하', '치료', '약', '없', '자폐증', '주목하', '전문', '치료제', '개발하', '이', '회사', '설립하', '주', '뉴', '로벤', '티', '연구', '팀', '이온', '통로', '글루탐산', '수용체', '억제하', '치료제', '개발', '매진하', '억제제', '후보', '물질', '대하', '기술', '권리', '국내외', '확보하', '기전', '연구', '진행하', '또', '다양하', '자폐', '동물', '모델', '통하', '자폐', '동물', '사회', '사회', '선택', '측정하', '자폐', '범주', '성장', '치료제', '검색하', '시스템', '구축하', '자폐증', '대하', '다중', '작용', '점', '치료', '효능', '평가', '연구', '그', '결', '억제제', '자폐', '핵심', '증상', '사회', '장애', '반복', '과잉', '행동', '개선하', '경련

In [0]:
# Data Tokenizing
%tensorflow_version 2.x
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import math

validation_ratio = math.floor(len(texts) * 0.3)    	# 30%는 검증데이터로 사용한다. 나머지는 훈련데이터
max_words = 10000                                   # 데이터셋에서 가장 빈도 높은 10,000 개의 단어만 사용한다
maxlen = 200                                        # 200개 이후의 단어는 버린다
class_number = 9                                    # 분류할 클래스의 수

tokenizer = Tokenizer(num_words=max_words)  	      # 상위빈도 10,000 개의 단어만을 추려내는 Tokenizer 객체 생성
tokenizer.fit_on_texts(texts)                       # 단어 인덱스를 구축한다
word_index = tokenizer.word_index               	  # 단어 인덱스만 가져온다

In [0]:
# 전체 데이터가 가지고 있는 토큰의 수를 보여준다. texts_to_sequences()를 거쳐야 10,000개만 남는다. 

print('전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index))
print('word_index type: ', type(word_index))
print('word_index: ', word_index)

In [0]:
# Data Sequencing
# train 데이터를 Sequencing 한다
# 상위 빈도 10,000개의 단어를 word_index의 숫자 리스트로 변환. Tokenizer 결과가 여기서 반영된다
data = tokenizer.texts_to_sequences(texts) 

# maxlen의 수만큼으로 2D 텐서를 만든다
data = pad_sequences(data, maxlen=maxlen)

print('data:', data)

In [0]:
# One-Hot Encoding
# 원-핫 인코딩 함수
def to_one_hot(labels, dimension):
      results = np.zeros((len(labels), dimension))
      for i, label in enumerate(labels):
         results[i, label] = 1.
      return results


# 입력층에는 원-핫 인코딩을 수행하지 않는다
# data = to_one_hot(data, dimension=max_words)

# 출력층만 원-핫 인코딩을 수행한다 
labels = to_one_hot(labels, dimension=class_number)

In [0]:
# 출력층의 형태
print(labels)

In [0]:
# Train data와 Validation data 준비

print('데이터 텐서의 크기:', data.shape)        	# (90, 10000)
print('레이블 텐서의 크기:', labels.shape)      	# (90, 9) data와 label이 모두 2D 텐서가 되었음

indices = np.arange(data.shape[0])      		      # 0 ~ 89 까지의 숫자를 생성
np.random.shuffle(indices)                        # 0 ~ 89 까지의 숫자를 랜덤하게 섞음
data = data[indices]                              # 이것을 인덱스로 하여 2D 텐서 데이터를 섞음
labels = labels[indices]                          # label도 같은 순서로 섞음

x_train = data[validation_ratio:]                 # 훈련데이터의 70%를 훈련데이터
y_train = labels[validation_ratio:]               # 훈련데이터의 70%를 훈련데이터 Label
x_val = data[:validation_ratio]                   # 훈련데이터의 30%를 검증데이터
y_val = labels[:validation_ratio]                 # 훈련데이터의 30%를 검증데이터 Label

In [0]:
# 모델 정의하기
from tensorflow.keras import models
from tensorflow.keras import layers

embedding_dim = 50                              # 임베딩의 차원을 설정한다. 보통 200까지에서 적절히 설정한다. 
model = models.Sequential()                 		# 모델을 새로 정의

# 임베딩 층. 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해 input_length(=maxlen)를 지정. 출력층은 (input_dim, input_length, output_dim)가 됨
model.add(layers.Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen))
model.add(layers.Flatten())                     # 3D 임베딩 텐서를 (input_dim, input_length * output_dim) 크기의 2D 텐서로 펼친다

# 은닉층
model.add(layers.Dense(units=32, activation='relu'))

# 출력층. 출력 노드는 분류 개수인 9. 활성화 함수는 sigmoid와 성격이 같으나 다중분류에 사용되는 함수인 softmax 함수를 사용한다. 확률값을 출력
model.add(layers.Dense(units=class_number, activation='softmax'))

In [0]:
# 모델 요약 출력
model.summary()

In [0]:
# Compile Model
# 가중치 업데이트는 이동평균 방법인 RMSprop을 사용하였다.
# 신경망의 출력이 확률이므로 crossentropy를 사용하는 것이 최선이다
# crossentropy는 원본의 확률 분포와 예측의 확률 분포를 측정하여 조절해 간다
# 또한 다중 분류이므로 categorical_crossentropy를 사용한다

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
# Train Model
# 32개씩 미니 배치를 만들어 10번의 epoch로 훈련
# 훈련데이터로 훈련하고, 검증데이터로 검증한다
# 반환값의 history는 훈련하는 동안 발생한 모든 정보를 담고 있는 딕셔너리이다

history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

history_dict = history.history

In [0]:
# 경로 변경
cd /content/gdrive/My Drive/pytest/

In [0]:
# multidimensional numpy arrays를 저장할 수 있는 h5 file(HDF) 포맷으로 저장한다
model.save('text_multi_we_model.h5') 	     # 모델 저장

# 훈련데이터에서 사용된 상위빈도 10,000개의 단어로 된 Tokenizer 저장
# 새로 입력되는 문장에서도 같은 단어가 추출되게 한다
import pickle
with open('text_multi_we_tokenizer.pickle', 'wb') as handle:
     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# Accuracy and Loss 확인
# history 딕셔너리 안에 있는 정확도와 손실값을 가져와 본다
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

print('Accuracy of each epoch:', acc)
epochs = range(1, len(acc) +1)

In [0]:
# Plotting Accuracy
# 정확도와 손실값의 변화를 보고, epoch를 어디에서 조절해야 할 지를 가늠한다.
# 정확도가 떨어지는 구간, 손실값이 높게 나타나는 구간을 확인한다
# 데이터가 큰 경우 대개 epoch를 늘려야 최적값에 도달한다
import matplotlib.pyplot as plt

# 정확도 그리기
plt.plot(epochs, acc, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()            			

In [0]:
# Plotting Loss
plt.figure()            		# 새로운 그림을 그린다

# 손실값 그리기
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()            		

plt.show()

In [0]:
# Load Model
import os
from tensorflow.keras.models import load_model

filepath = '/content/gdrive/My Drive/pytest/'
os.chdir(filepath)
print("Current Directory:", os.getcwd())

loaded_model = load_model('text_multi_we_model.h5')
print("model loaded:", loaded_model)

with open('text_multi_we_tokenizer.pickle', 'rb') as handle:
       loaded_tokenizer = pickle.load(handle)

In [0]:
# Test Data Loading
# 테스트 데이터의 텍스트와 라벨을 받아온다

texts, labels = make_texts_labels(test_dir, label_types)

print('texts 0:', texts[0])
print('texts len:', len(texts))
print('labels 0:', labels[0])
print('labels len:', len(labels))

In [0]:
# 로딩된 tokenizer로 test 데이터를 Sequencing 한다
data = loaded_tokenizer.texts_to_sequences(texts) 	# 문자열을 숫자 리스트로 변환
x_test = pad_sequences(data, maxlen=maxlen)     	    # maxlen의 수만큼으로 2D 텐서를 만듦
print('data:', data)


# label에 해당하는 부분만 one-hot-encoding 한다.
# x_test = to_one_hot(data, dimension=max_words)
y_test = to_one_hot(labels, dimension=class_number)

In [0]:
# Test Data Evaluation
test_eval = loaded_model.evaluate(x_test, y_test)
print('prediction model loss & acc:', test_eval)			# 44.4% 까지 나왔다

In [0]:
predictions = loaded_model.predict(x_test)
print('predictions shape:', predictions.shape)
print('prediction 0 shape:', predictions[0].shape)
print('prediction 0 sum', np.sum(predictions[0]))             # 모든 분류에 대한 확률의 총합은 1이 되어야 한다

print('prediction 0 value:', predictions[0])                  # 9개 분류 각각에 대한 확률값
print('prediction 0 max value:', np.argmax(predictions[0]))  	# prediction 0이 가지고 있는 가장 큰 값의 판정 결과
print('prediction 1 max value:', np.argmax(predictions[1]))   # prediction 1이 가지고 있는 가장 큰 값의 판정 결과
print('prediction 2 max value:', np.argmax(predictions[2]))   # prediction 2가 가지고 있는 가장 큰 값의 판정 결과

In [0]:
# 1개 데이터 예측
text = ["안녕하세요, 이것은 테스트 문장입니다."]       	# list 타입으로 만든다

text=[rhinoMorph.onlyMorph_list(rn, sentence, pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'], eomi=False) for sentence in text]
print('형태소 분석 결과:', text)

data = loaded_tokenizer.texts_to_sequences(text)
x_test = pad_sequences(data, maxlen=maxlen)

predictions = loaded_model.predict(x_test)
print("Result:", predictions)                           # 각 분류에 대한 확률을 보여준다
print("분류 결과:", np.argmax(predictions[0]))         	# 가장 큰 확률값을 가진 분류 제시. 0~8 숫자 중 하나